In [24]:
import json
from tqdm import tqdm
import os
import re

In [25]:
# lets know where we are
cwd = os.getcwd()
print(cwd)

/home/ubuntu/cvd19-truth-finder/notebooks


# Retrieiver

In [ ]:
RAW_DOC_PATH = "../cvd19-documents-raw/biorxiv_medrxiv/biorxiv_medrxiv"
PROCESSED_DOC_PATH = "../data/covid19/pre-database-documents/"

# Create Pre-Processed Files from papers

In [27]:
def iter_files(path):
    """Walk through all files located under a root path."""
    if os.path.isfile(path):
        yield path
    elif os.path.isdir(path):
        for dirpath, _, filenames in os.walk(path):
            for f in filenames:
                yield os.path.join(dirpath, f)
    else:
        raise RuntimeError('Path %s is invalid' % path)
        
def collect_text(text):
    """Combing list of text elements into one."""
    return " ".join(regex_filter(seg['text']) for seg in text)

def regex_filter(text):
    """Removing numbers and puncutation."""
    text = re.sub(r" \d+", "",text)
    return re.sub(r"[^A-Za-z0-9 -]+", "",text)

In [34]:
pre_processed_filename = "biorxiv_medrxiv.json"

# get all json papers in directory
jsons = list(iter_files(RAW_DOC_PATH))

# process them so they are {"id", <ID>, "text", <TEXT>} as specified here: https://github.com/facebookresearch/DrQA/tree/master/scripts/retriever
with open(os.path.join(PROCESSED_DOC_PATH, pre_processed_filename),  "w") as f:
    for j in tqdm(jsons):
        
        # reading in document
        document = json.loads(open(j).read())
        
        # extracting id and abstract
        doc_record = {
            "id": document['paper_id'],
            "text": collect_text(document['abstract'])
        }
        
        # saving to file
        f.write(json.dumps(doc_record)+"\n")
        
        

100%|██████████| 4/4 [00:00<00:00, 1058.97it/s]


# Create an SQLitte Database from Pre-Processed Files

In [36]:
!python ../scripts/retriever/build_db.py \
"/home/ubuntu/cvd19-truth-finder/data/covid19/pre-database-documents/biorxiv_medrxiv.json" \
"/home/ubuntu/cvd19-truth-finder/data/covid19/doc-db/doc.db" \
--num-workers=1

/home/ubuntu/cvd19-truth-finder/notebooks
/home/ubuntu/cvd19-truth-finder/data/covid19/doc-db/doc.db
['.ipynb_checkpoints']
05/09/2020 02:02:57 PM: [ Reading into database... ]
  0%|                                                     | 0/1 [00:00<?, ?it/s]
1it [00:00, 1082.68it/s]
100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 552.25it/s]
05/09/2020 02:02:57 PM: [ Read 4 docs. ]
05/09/2020 02:02:57 PM: [ Committing... ]


# Create TF-IDF Model from Database

In [37]:
!python ../scripts/retriever/build_tfidf.py \
"/home/ubuntu/cvd19-truth-finder/data/covid19/doc-db/doc.db" \
"/home/ubuntu/cvd19-truth-finder/data/covid19/doc_ranker"

05/09/2020 02:16:47 PM: [ Counting words... ]
05/09/2020 02:16:47 PM: [ Mapping... ]
05/09/2020 02:16:47 PM: [ -------------------------Batch 1/4------------------------- ]
05/09/2020 02:16:47 PM: [ -------------------------Batch 2/4------------------------- ]
05/09/2020 02:16:47 PM: [ -------------------------Batch 3/4------------------------- ]
05/09/2020 02:16:47 PM: [ -------------------------Batch 4/4------------------------- ]
05/09/2020 02:16:47 PM: [ Creating sparse matrix... ]
05/09/2020 02:16:47 PM: [ Making tfidf vectors... ]
05/09/2020 02:16:49 PM: [ Getting word-doc frequencies... ]
05/09/2020 02:16:50 PM: [ Saving to /home/ubuntu/cvd19-truth-finder/data/covid19/doc_ranker/doc-tfidf-ngram=2-hash=16777216-tokenizer=simple.npz ]


---

# Reader

### Pre-Process SQUAD Dataset

#### Train set

In [55]:
!python ../scripts/reader/preprocess.py \
"/home/ubuntu/cvd19-truth-finder/drQA-training-datasets/datasets" \
"/home/ubuntu/cvd19-truth-finder/drQA-training-datasets/datasets" \
--split SQuAD-v1.1-train \
--tokenizer spacy

Loading dataset /home/ubuntu/cvd19-truth-finder/drQA-training-datasets/datasets/SQuAD-v1.1-train.json
Will write to file /home/ubuntu/cvd19-truth-finder/drQA-training-datasets/datasets/SQuAD-v1.1-train-processed-spacy.txt
Total time: 142.2876 (s)


#### dev set

In [57]:
!python ../scripts/reader/preprocess.py \
"/home/ubuntu/cvd19-truth-finder/drQA-training-datasets/datasets" \
"/home/ubuntu/cvd19-truth-finder/drQA-training-datasets/datasets" \
--split SQuAD-v1.1-dev \
--tokenizer spacy

Loading dataset /home/ubuntu/cvd19-truth-finder/drQA-training-datasets/datasets/SQuAD-v1.1-dev.json
Will write to file /home/ubuntu/cvd19-truth-finder/drQA-training-datasets/datasets/SQuAD-v1.1-dev-processed-spacy.txt
Total time: 21.2823 (s)


## Train Model

In [67]:
!python ../scripts/reader/train.py \
--num-epochs=1 \
--model-dir="/home/ubuntu/cvd19-truth-finder/data/covid19/model" \
--data-dir="/home/ubuntu/cvd19-truth-finder/drQA-training-datasets/datasets" \
--train-file="/home/ubuntu/cvd19-truth-finder/drQA-training-datasets/datasets/SQuAD-v1.1-train-processed-spacy.txt" \
--dev-file="/home/ubuntu/cvd19-truth-finder/drQA-training-datasets/datasets/SQuAD-v1.1-dev-processed-spacy.txt" \
--embed-dir="/home/ubuntu/cvd19-truth-finder/drQA-training-datasets/datasets/" \
--embedding-file="glove.6B.50d.txt"

05/09/2020 04:41:20 PM: [ COMMAND: ../scripts/reader/train.py --num-epochs=1 --model-dir=/home/ubuntu/cvd19-truth-finder/data/covid19/model --data-dir=/home/ubuntu/cvd19-truth-finder/drQA-training-datasets/datasets --train-file=/home/ubuntu/cvd19-truth-finder/drQA-training-datasets/datasets/SQuAD-v1.1-train-processed-spacy.txt --dev-file=/home/ubuntu/cvd19-truth-finder/drQA-training-datasets/datasets/SQuAD-v1.1-dev-processed-spacy.txt --embed-dir=/home/ubuntu/cvd19-truth-finder/drQA-training-datasets/datasets/ --embedding-file=glove.6B.50d.txt ]
05/09/2020 04:41:20 PM: [ ---------------------------------------------------------------------------------------------------- ]
05/09/2020 04:41:20 PM: [ Load data files ]
05/09/2020 04:41:41 PM: [ Num train examples = 86600 ]
05/09/2020 04:41:43 PM: [ Num dev examples = 10570 ]
05/09/2020 04:41:43 PM: [ ---------------------------------------------------------------------------------------------------- ]
05/09/2020 04:41:43 PM: [ Training mod

/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:543: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:543: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:543: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:543: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:543: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,plea

/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:543: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:543: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:543: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:543: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:543: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,plea

/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:543: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:543: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:543: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:543: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:543: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,plea

/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:543: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:543: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:543: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:543: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:543: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,plea

...

/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:543: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:543: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:543: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:543: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:543: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,plea

/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:543: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:543: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:543: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:543: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:543: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,plea

/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:543: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:543: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:543: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:543: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:543: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,plea

/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:543: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:543: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:543: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:543: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:543: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,plea

/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:543: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:543: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:543: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:543: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:543: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,plea

# Running Full Pipeline

In [16]:
import torch
import argparse
import code
import prettytable

from termcolor import colored
from drqa import pipeline
from drqa.retriever import utils

if __name__ == '__main__':
    # Arguments
    args_reader_model="/home/ubuntu/cvd19-truth-finder/data/covid19/model/20200509-c9d21e14.mdl"
    args_retriever_model="/home/ubuntu/cvd19-truth-finder/data/covid19/doc_ranker/doc-tfidf-ngram=2-hash=16777216-tokenizer=simple.npz"
    args_doc_db="/home/ubuntu/cvd19-truth-finder/data/covid19/doc-db/doc.db"
    args_tokenizer="spacy"
    args_candidate_file=None
    args_no_cuda=True
    args_gpu=-1

    args_cuda = not args_no_cuda and torch.cuda.is_available()
    if args_cuda:
        torch.cuda.set_device(args_gpu)

    if args_candidate_file:
        candidates = set()
        with open(args_candidate_file) as f:
            for line in f:
                line = utils.normalize(line.strip()).lower()
                candidates.add(line)
    else:
        candidates = None

    DrQA = pipeline.DrQA(
        cuda=args_cuda,
        fixed_candidates=candidates,
        reader_model=args_reader_model,
        ranker_config={'options': {'tfidf_path': args_retriever_model}},
        db_config={'options': {'db_path': args_doc_db}},
        tokenizer=args_tokenizer
    )

    # ------------------------------------------------------------------------------
    # Drop in to interactive mode
    # ------------------------------------------------------------------------------


    def process(question, candidates=None, top_n=1, n_docs=5):
        predictions = DrQA.process(
            question, candidates, top_n, n_docs, return_context=True
        )
        table = prettytable.PrettyTable(
            ['Rank', 'Answer', 'Doc', 'Answer Score', 'Doc Score']
        )
        for i, p in enumerate(predictions, 1):
            table.add_row([i, p['span'], p['doc_id'],
                           '%.5g' % p['span_score'],
                           '%.5g' % p['doc_score']])
        print('Top Predictions:')
        print(table)
        print('\nContexts:')
        for p in predictions:
            text = p['context']['text']
            start = p['context']['start']
            end = p['context']['end']
            output = (text[:start] +
                      colored(text[start: end], 'green', attrs=['bold']) +
                      text[end:])
            print('[ Doc = %s ]' % p['doc_id'])
            print(output + '\n')

    process("Who is infected?")

processing paragraph: 0


/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:543: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:543: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:543: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:543: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
Process ForkPoolWorker-16:
Process ForkPoolWorker-19:
Process ForkPoolWorker-18:
Process ForkPoolWorker-23:
Process ForkPoolWorker-21:
Process ForkPoolWorker-20:
Pr

KeyboardInterrupt: 

  File "/home/ubuntu/anaconda3/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/home/ubuntu/anaconda3/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
